<a href="https://colab.research.google.com/github/Shahahaha11/LION_DEN/blob/Chi's-Branch/xg_boost_trial_RFE_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
df_2 = pd.read_pickle("/content/drive/MyDrive/lions_den_data/filtered_features_RFE.pkl")
df_2.columns

Index(['longitude', 'latitude', 'numOfPhotos', 'livingAreaSqFt',
       'avgSchoolRating', 'numOfBathrooms', 'Income', 'MORTGAGE30US',
       'MEDLISPRIPERSQUFEE12420', 'hasAssociation', 'latestPrice',
       'latest_saledate'],
      dtype='object')

In [ ]:
df_2 = df_2.drop(columns=['latest_saledate'])
df_2.columns

Index(['longitude', 'latitude', 'numOfPhotos', 'livingAreaSqFt',
       'avgSchoolRating', 'numOfBathrooms', 'Income', 'MORTGAGE30US',
       'MEDLISPRIPERSQUFEE12420', 'hasAssociation', 'latestPrice'],
      dtype='object')

In [ ]:
################ BASE XGB ##########################

import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
import xgboost as xgb
from sklearn.metrics import mean_squared_error, r2_score

# Assume df_2 is already loaded into your environment
# df_2 = pd.read_csv('your_file.csv')  # example loading method

# Define the target variable and features
target = 'latestPrice'
features = df_2.columns.drop(target)  # all columns except target

X = df_2[features]
y = df_2[target]

# Split the dataset into training and testing sets

X = df_2.drop(columns=['latestPrice'])

# FIRST SPLIT
n_train = int(0.8 * len(df_2))
train = df_2.iloc[:n_train]
test  = df_2.iloc[n_train:]

X_train = train.drop(columns=['latestPrice'])
y_train = train['latestPrice']
X_test = test.drop(columns=['latestPrice'])
y_test = test['latestPrice']


# Create a pipeline with scaling and XGBoost regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor(
        objective='reg:squarederror',  # standard objective for regression
        random_state=11,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        reg_alpha=0.1,  # L1 regularization
        reg_lambda=1.0  # L2 regularization
    ))
])

# Optionally, you can use GridSearchCV to tune hyperparameters including regularization parameters
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.05, 0.1, 0.2],
    'xgb__max_depth': [3, 5, 7],
    'xgb__reg_alpha': [0, 0.1, 0.5],
    'xgb__reg_lambda': [0.5, 1.0, 2.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Fit the model on training data
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
print("Best parameters found: ", grid_search.best_params_)

# Predict on test data
y_pred = grid_search.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
print(f"Test Mean Squared Error: {mse:.2f}")



Fitting 5 folds for each of 162 candidates, totalling 810 fits


In [ ]:
from sklearn.metrics import mean_squared_error, r2_score

mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test Mean Squared Error: {mse:.2f}")
print(f"Test R^2 Score: {r2:.2f}")

In [ ]:

y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)


train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Training Metrics:")
print(f"  R²:   {train_r2:.3f}")
print(f"  RMSE: {train_rmse:,.3f}")

print("\nValidation Metrics:")
print(f"  R²:   {test_r2:.3f}")
print(f"  RMSE: {test_rmse:,.3f}")

Training Metrics:
  R²:   0.878
  RMSE: 156,522.705

Validation Metrics:
  R²:   0.660
  RMSE: 276,054.394


In [ ]:
###################### TRYTING XGB WITH GWR ################### GWR using mgwr library: MAIN : output -> local_betas ############

import pandas as pd
import numpy as np
from mgwr.gwr import GWR
from mgwr.sel_bw import Sel_BW

coords = X_train[['latitude', 'longitude']].values
# dropping : garageSpaces and parkingSpaces are perfectly collinear (their correlation is 1 and their VIFs are infinite)
X_ = X_train.select_dtypes(include=['number']).drop(columns=['latitude', 'longitude', 'latest_saledate', 'parkingSpaces', 'MORTGAGE30US', 'MEDLISPRIPERSQUFEE12420'],
    errors='ignore'
).values

y_ = y_train.values.reshape(-1, 1)
X_gwr = np.hstack([np.ones((X_.shape[0], 1)), X_])

# using a fixed band width
fixed_bw = 0.3

gwr_model = GWR(
    coords,       # (n, 2)
    y_,           # (n, 1)
    X_gwr,        # (n, p) [including intercept]
    bw=fixed_bw,
    kernel='gaussian',  # Gaussian kernel
    fixed=True,         # fixed distance bandwidth
    spherical=False
).fit()

local_betas = gwr_model.params  # shape (n, p)
print("Local betas shape:\n", local_betas.shape)


Local betas shape:
 (12016, 30)


In [ ]:
############################################### XGB with Local Slopes #################################################
# MODEL 0 "USING local_betas"

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, r2_score


X_xgb = X_train.select_dtypes(include=['number']).drop(
    columns=['latitude', 'longitude', 'latest_saledate', 'parkingSpaces'],
    errors='ignore'
).values
print("X_xgb shape:", X_xgb.shape)

# local_betas are obtained from the GWR model; its shape is (n, p).
local_betas = gwr_model.params
print("local_betas shape:", local_betas.shape)

# local_slopes are defined by dropping the intercept column (first column) from local_betas.
# This gives us an array of shape (n, p-1).
local_slopes = local_betas[:, 1:]

X_aug = np.hstack([X_xgb, local_slopes])
print("Augmented feature shape:", X_aug.shape)

y_array = y_.ravel()


n_total = X_aug.shape[0]
n_train = int(0.8 * n_total)

X_train = X_aug[:n_train]
y_train = y_array[:n_train]
X_test = X_aug[n_train:]
y_test = y_array[n_train:]

# Create a pipeline with scaling and XGBoost regressor
pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('xgb', xgb.XGBRegressor(
        objective='reg:squarederror',  # standard objective for regression
        random_state=11,
        n_estimators=100,
        learning_rate=0.1,
        max_depth=5,
        reg_alpha=0.1,  # L1 regularization
        reg_lambda=1.0  # L2 regularization
    ))
])

# Optionally, you can use GridSearchCV to tune hyperparameters including regularization parameters
param_grid = {
    'xgb__n_estimators': [100, 200],
    'xgb__learning_rate': [0.05, 0.1, 0.2],
    'xgb__max_depth': [3, 5, 7],
    'xgb__reg_alpha': [0, 0.1, 0.5],
    'xgb__reg_lambda': [0.5, 1.0, 2.0]
}

grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring='neg_mean_squared_error',
    verbose=1,
    n_jobs=-1
)

# Fit the model on training data
grid_search.fit(X_train, y_train)

# Get the best model and its parameters
print("Best parameters found: ", grid_search.best_params_)

# Predict on test data
y_pred = grid_search.predict(X_test)

# Evaluate model performance
mse = mean_squared_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)
print(f"Test Mean Squared Error: {mse:.2f}")
print(f"Test R^2 Score: {r2:.2f}")

X_xgb shape: (12016, 31)
local_betas shape: (12016, 30)
Augmented feature shape: (12016, 60)
Fitting 5 folds for each of 162 candidates, totalling 810 fits
Best parameters found:  {'xgb__learning_rate': 0.2, 'xgb__max_depth': 3, 'xgb__n_estimators': 100, 'xgb__reg_alpha': 0, 'xgb__reg_lambda': 2.0}
Test Mean Squared Error: 57102795916.00
Test R^2 Score: 0.68


In [ ]:
################## CHECK OVER FITTING ##########################

y_train_pred = grid_search.predict(X_train)
y_test_pred = grid_search.predict(X_test)


train_r2 = r2_score(y_train, y_train_pred)
test_r2 = r2_score(y_test, y_test_pred)


train_rmse = np.sqrt(mean_squared_error(y_train, y_train_pred))
test_rmse = np.sqrt(mean_squared_error(y_test, y_test_pred))

print("Training Metrics:")
print(f"  R²:   {train_r2:.3f}")
print(f"  RMSE: {train_rmse:,.3f}")

print("\nValidation Metrics:")
print(f"  R²:   {test_r2:.3f}")
print(f"  RMSE: {test_rmse:,.3f}")

Training Metrics:
  R²:   0.867
  RMSE: 165,379.131

Validation Metrics:
  R²:   0.680
  RMSE: 238,961.913
